In [1]:
# IMPORTAÇÕES
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import unidecode
import Database
import json
from datetime import datetime

In [2]:
#Carregar arquivo na memória
db = Database.Database('CNPq')
engine = Carga.db_engine()
dt = pd.read_sql("indicadores_pareamento", engine)
print(f'dt.size: {dt.size}')
#size: 386.669.844      


dt.size: 386669844


In [3]:
#Pegando lista de pareamentos jpa realizados
ids_já_feitos = []
sql = 'SELECT DISTINCT id FROM pareamentos;'
ids_já_feitos = db.query(sql)
ids_já_feitos = [num[0] for num in ids_já_feitos] # Removendo lista dentro de lista por recuperar SQL
print(f'len(ids_já_feitos): {len(ids_já_feitos)}')

len(ids_já_feitos): 11857


In [4]:
## Removendo os ids já feitos
dt = dt[~dt.id.isin(ids_já_feitos)]

In [5]:
## Retirada dos não doutores - Existem 3 não doutores contemplados!!!
dt = dt[pd.notnull(dt.ano_doutorado)]


## Normalizando as Tabelas

#Preenchendo Nan com zeros nos indicadores
dt.qty_2012 = dt.qty_2012.fillna(0)
dt.qty_2013 = dt.qty_2013.fillna(0)
dt.qty_2014 = dt.qty_2014.fillna(0)
dt.qty_2015 = dt.qty_2015.fillna(0)
dt.qty_2016 = dt.qty_2016.fillna(0)
dt.qty_2017 = dt.qty_2017.fillna(0)
dt.qty_2018 = dt.qty_2018.fillna(0)
dt.qty_2019 = dt.qty_2019.fillna(0)

#Preenchendo Nan com "nenhum" nas áreas e vínculos
dt.area_demanda_bruta = dt.area_demanda_bruta.fillna('nenhum')
dt.areas_conhecimento = dt.areas_conhecimento.fillna('nenhum')
dt.tipos_vinculo = dt.tipos_vinculo.fillna('nenhum')
dt.enquadramento_vinculo = dt.enquadramento_vinculo.fillna('nenhum')

#Removendo acentos e caracteres especiais. Colocando tudo em minúsculas.
dt.area_demanda_bruta = dt.area_demanda_bruta.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.areas_conhecimento = dt.areas_conhecimento.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.tipos_vinculo = dt.tipos_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()
dt.enquadramento_vinculo = dt.enquadramento_vinculo.str.normalize('NFKD')\
       .str.encode('ascii', errors='ignore')\
       .str.decode('utf-8')\
       .str.lower()              

## Criando tabelas parciais
financiados = dt[~pd.isnull(dt.pgtos)]
dt = dt[pd.isnull(dt.pgtos)]
#erros = dt[dt.id.isin(financiados.id.unique())] #-> Listagem dos erros encontrados

## Alguns pesquisadores foram financiados em um ano, mas não em outro. Assim, aparece duas vezes, uma com pgtos=Nan, outra com pgtos > 0.
## Assim, é necessário retirar da lista de pareamento aqueles que foram financiados em algum momento.
## Da mesma forma, com certeza houveram aqueles financiados mais de um ano.
dt = dt[~dt.id.isin(financiados.id.unique())] #Percebi alguns erros, então tive a certeza de remover os financiados da tabela de pareamento




In [14]:
#
#
#
## Fazendo Pareamento
#
#
#

min_para_parear = 3 # -> número de pessoas pareadas de modo fixo, sem usar os erros
num_pareamentos = 3 # -> número de pessoas pareadas ao final -> vai usar erros quadráticos para chegar nesse número

lista_pareamentos = [] # -> Lista com todos os pareados -> Útil apenas para conferência depois
lista_pareamentos_describe = []  #  -> lista com os describe de todos os pareamentos

número_ids_total = financiados.id.unique().size
número_ids_já_feitos = 0
tempo_início = datetime.now()

for id in financiados.id.unique():
    print(f'{número_ids_já_feitos}/{número_ids_total}. Fazendo id: {id}. Iniciado em {tempo_início}')
    if not número_ids_já_feitos == 0:
        porcentagem_já_feita = (número_ids_já_feitos/número_ids_total)
        tempo_passado = datetime.now() - tempo_início
        tempo_por_id = tempo_passado / número_ids_já_feitos
        tempo_restante = (número_ids_total - número_ids_já_feitos) * tempo_por_id
        tempo_em_que_vai_acabar = datetime.now() + tempo_restante
        print(f'{número_ids_já_feitos}. {porcentagem_já_feita * 100}% feitos. Fazendo id: {id}. Acabará em {tempo_em_que_vai_acabar}')
    número_ids_já_feitos += 1

    #Pegando dados do ID
    financiado = financiados[financiados.id == id]
    
    #Pegando áreas do Conhecimento do Financiado
    financiado.areas_conhecimento.iloc[0].split(',')
    
    #Pegando Regiões do Financiado
    região = financiado.iloc[0].uf
    tipo_região = None
    if not região == None:
        if região in ('SP'):
            tipo_região = 1
        elif região in ('MG', 'RS', 'RJ', 'PR'):
            tipo_região = 2
        else:
            tipo_região = 3
    
    #Pegando ano do Indicador a ser usado do Financiado -> Útil para calcular erro -> Vai ser usado ao calcular o impacto
    if financiado.chamada.str.contains('2012', na=False).unique()[0] == True : ano = 2012
    elif financiado.chamada.str.contains('2013', na=False).unique()[0] == True : ano = 2013
    elif financiado.chamada.str.contains('2014', na=False).unique()[0] == True : ano = 2014
    else: ano = None

    #Pegando Faixa do Financiado do Financiado -> Inútil aqui -> Vai ser usado ao calcular o impacto
    if financiado.chamada.str.contains('Faixa A', na=False).unique()[0] == True : faixa = 'A'
    elif financiado.chamada.str.contains('Faixa B', na=False).unique()[0] == True : faixa = 'B'
    elif financiado.chamada.str.contains('Faixa C', na=False).unique()[0] == True : faixa = 'C'
    else: faixa = None
        
    #Pegando o vínculo do Financiado
    tipos_vinculo = financiado.tipos_vinculo.unique()
    if not tipos_vinculo == None and len(tipos_vinculo) > 0:   
        if tipos_vinculo[0].find('servidor_publico') > -1:
            é_servidor = 'servidor_publico'
        else:
            é_servidor = 'outros'   
    else: é_servidor = 'nenhum'
    
    #   
    #Fazendo o Pareamento
    #
    tipos_de_pareamento = []  #Variável que vai indicar quais os tipos de pareamento realizados
    pareados = dt
    
    #1. Pareando pela área de conhecimento
    print('    1. Pareando pela área de conhecimento')
    pareados2 = pareados.loc[(pareados.areas_conhecimento.str.contains(financiado.area_demanda_bruta.unique()[0].split('(')[0].strip(), na=False))]
    if pareados2.id.unique().size > min_para_parear:
        tipos_de_pareamento.append('area_demanda_bruta')
        pareados = pareados2
                       
    #2. Pareando pelo Vínculo
    print('    2. Pareando pelo Vínculo')
    if é_servidor == 'servidor_publico':
        pareados2 = pareados.loc[(pareados.tipos_vinculo.str.contains('servidor_publico', na=False))]
    else:
        pareados2 = pareados.loc[~(pareados.tipos_vinculo.str.contains('servidor_publico', na=False))]
    if pareados2.id.unique().size > min_para_parear:
        pareados = pareados2
        tipos_de_pareamento.append('tipos_vinculo')
                                             
    #6. Pareando por região
    print('    6. Pareando por região')
    if tipo_região == 1:
        pareados2 = pareados.loc[pareados.uf.isin(('SP',))]
    elif tipo_região == 2:
        pareados2 = pareados.loc[pareados.uf.isin(('MG', 'RS', 'RJ', 'PR'))]
    elif tipo_região == 3:
        pareados2 = pareados.loc[~pareados.uf.isin(('SP', 'MG', 'RS', 'RJ', 'PR'))]
    if pareados2.id.unique().size > min_para_parear:
        pareados = pareados2
        tipos_de_pareamento.append('uf')
                       
    #7. Pareando por erros quadráticos
    tipos_de_pareamento.append('erro')
                                            
    #Fezendo lista de erros quadráticos
    ids_pareados = []
    if not pareados.empty:

        #Calculando os erros dos grupos
        # "grupo" é um dos grupos de indicadores.
        for grupo in financiado.grupo.unique():
            print(f'    7. Pareando por erros quadráticos. Grupo: {grupo}. Qty: {len(pareados.id.unique())}')
            pareados_grupo = pareados.loc[(pareados.grupo == grupo)]


            #Pegando o valor do Financiado
            #print('        Pegando o valor do Financiado')
            ind_financiado = financiado.loc[(financiado.grupo == grupo)]['qty_' + str(ano)].iloc[0]

            #Verificando o erro dos pareados restantes
#            mum_erro2_0 = 0
            for id_pareado in pareados.id.unique():
                ind_pareado = pareados_grupo.loc[(pareados.id == id_pareado)]['qty_' + str(ano)]
                if len(ind_pareado) > 0:
                    ind_pareado = ind_pareado.iloc[0]
                else:
                    ind_pareado = 0
                erro2 = (ind_financiado - ind_pareado)**2
                erro = {
                    'id': id,
                    'id_pareado': id_pareado,
                    'grupo': grupo,
                    'valor': ind_pareado,
                    'erro2': erro2
                }
#                if erro2 == 0:
#                    mum_erro2_0 += 1
#                if mum_erro2_0 > 4:
#                    break
                ids_pareados.append(erro)
                #print(f'        Verificando o id apreado {id_pareado} com erro', erro['erro2'], '.')

        #Acrescentando Anos de Doutor como um Indicador
        print(f'    7. Pareando por erros quadráticos. Grupo: Anos de Doutor.')
        for id_pareado in pareados.id.unique():
            ind_pareado = pareados.ano_doutorado.loc[
                (pareados.id == id_pareado)
                ].unique()
            #print(ind_pareado, type(ind_pareado))
            if len(ind_pareado) > 0:
                ind_pareado = ind_pareado[0]
            else:
                ind_pareado = 0
            erro = {
                'id': id,
                'id_pareado': id_pareado,
                'grupo': "Anos de Doutor",
                'valor': ind_pareado,
                'erro2': (financiado.ano_doutorado.iloc[0] - ind_pareado)**2
            }
            ids_pareados.append(erro)

    #Calculando o Desvio Padrão por id pareado
    print('    Calculando o Desvio Padrão por id pareado')
    erro = pd.DataFrame(ids_pareados)
    desvio_padrão = []
    if not erro.empty:
        desvio_padrão = []  # -> VARIÁVEL QUE VAI CONTER OS IDS PAREADOS
        for id_pareado in erro.id_pareado.unique():
            erro_id = erro.loc[erro.id_pareado == id_pareado]
            if not erro_id.empty:
                soma_erro = erro_id.erro2.sum()
                num_erros = len(erro_id.index)
                err = {
                    'id_pareado': id_pareado,
                    'erro': np.sqrt(soma_erro/num_erros),
                }
                
                desvio_padrão.append(err)
                lista_pareamentos.append(err)

                #db.insert_dict("pareamentos", err, on_conflict=['id', 'id_pareado'])
        pareados_por_desvio_padrão = pd.DataFrame(desvio_padrão)
        if not pareados_por_desvio_padrão.empty:
            lista_pareados = pareados_por_desvio_padrão.sort_values(by=['erro']).iloc[:num_pareamentos]
            lista_pareamentos_describe.append(pareados_por_desvio_padrão.describe().to_json())
            #print(pareados_por_desvio_padrão.describe())
            #print(lista_pareados)
                      
    lista_pareado_para_incluir_no_bd = {
        'id': id,
        'ano': ano,
        'faixa': faixa,
        'tipos_pareamento': json.dumps(tipos_de_pareamento),
        'pareamento_describe': json.dumps(pareados_por_desvio_padrão.describe().to_json())
    }
    x = 1
    for pareado in desvio_padrão:
        lista_pareado_para_incluir_no_bd['pareado_' + str(x)] = pareado['id_pareado']
        lista_pareado_para_incluir_no_bd['erro_' + str(x)] = pareado['erro']
        x += 1
        if x > num_pareamentos: break
    print('    Incluindo no DB.')
    db.insert_dict('pareamentos', lista_pareado_para_incluir_no_bd, on_conflict=['id'])
    #print(lista_pareado_para_incluir_no_bd)
    
porcentagem_já_feita = (número_ids_já_feitos/número_ids_total)
print(f'{número_ids_já_feitos}. {porcentagem_já_feita * 100}% feitos. Último feito: {id}. Acabou em {datetime.now()}.')


0.Fazendo id: 6813932799267174 / 716. Iniciado em 2022-04-18 18:37:08.497572
    1. Pareando pela área de conhecimento
    2. Pareando pelo Vínculo
    6. Pareando por região
    7. Pareando por erros quadráticos. Grupo: Apresentações de Trabalhos. Qty: 16719
    7. Pareando por erros quadráticos. Grupo: Atividades de Conselho, Direção ou Administração. Qty: 16719
    7. Pareando por erros quadráticos. Grupo: Atividades de Ensino. Qty: 16719
    7. Pareando por erros quadráticos. Grupo: Atividades de Pesquisa e Desenvolvimento. Qty: 16719
    7. Pareando por erros quadráticos. Grupo: Doutorados Iniciados. Qty: 16719
    7. Pareando por erros quadráticos. Grupo: Eventos. Qty: 16719
    7. Pareando por erros quadráticos. Grupo: Graduações Iniciadas. Qty: 16719
    7. Pareando por erros quadráticos. Grupo: Livros Publicados. Qty: 16719
    7. Pareando por erros quadráticos. Grupo: Mestrados. Qty: 16719
    7. Pareando por erros quadráticos. Grupo: Orientações de Mestrado. Qty: 16719
    7

    7. Pareando por erros quadráticos. Grupo: Cursos. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Doutorados Iniciados. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Eventos. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Graduações Iniciadas. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Livros Publicados. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Mestrados. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Mídias. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Orientações de Doutorado. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Outras Pós-Graduações lato sensu. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Pariticipação em Bancas - Outras. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Pariticipação em Bancas de Graduação. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Pariticipação em Bancas de Pós-Graduação. Qty: 71
    7. Pareando por erros quadráticos. Grupo: Participação em Congressos. Qty: 

    Calculando o Desvio Padrão por id pareado
    Incluindo no DB.
8.Fazendo id: 6825932780256380 / 716. Iniciado em 2022-04-18 18:37:08.497572
8. 1.1173184357541899% feitos. Fazendo id: 6825932780256380. Acabará em 2022-04-20 11:42:38.462310
    1. Pareando pela área de conhecimento
    2. Pareando pelo Vínculo
    6. Pareando por região
    7. Pareando por erros quadráticos. Grupo: Apresentações de Trabalhos. Qty: 133
    7. Pareando por erros quadráticos. Grupo: Atividades de Conselho, Direção ou Administração. Qty: 133
    7. Pareando por erros quadráticos. Grupo: Atividades de Ensino. Qty: 133
    7. Pareando por erros quadráticos. Grupo: Atividades de Pesquisa e Desenvolvimento. Qty: 133
    7. Pareando por erros quadráticos. Grupo: Cursos. Qty: 133
    7. Pareando por erros quadráticos. Grupo: Doutorados Iniciados. Qty: 133
    7. Pareando por erros quadráticos. Grupo: Eventos. Qty: 133
    7. Pareando por erros quadráticos. Grupo: Graduações Iniciadas. Qty: 133
    7. Pareando 

    7. Pareando por erros quadráticos. Grupo: Premiações. Qty: 99
    7. Pareando por erros quadráticos. Grupo: Produções Bibliográficas. Qty: 99
    7. Pareando por erros quadráticos. Grupo: Produções Técnicas. Qty: 99
    7. Pareando por erros quadráticos. Grupo: Publicações de Artigos. Qty: 99
    7. Pareando por erros quadráticos. Grupo: Vínculos Empregatícios ou Funcionais. Qty: 99
    7. Pareando por erros quadráticos. Grupo: Anos de Doutor.
    Calculando o Desvio Padrão por id pareado
    Incluindo no DB.
12.Fazendo id: 6829909822082760 / 716. Iniciado em 2022-04-18 18:37:08.497572
12. 1.675977653631285% feitos. Fazendo id: 6829909822082760. Acabará em 2022-04-19 22:23:52.621397
    1. Pareando pela área de conhecimento
    2. Pareando pelo Vínculo
    6. Pareando por região
    7. Pareando por erros quadráticos. Grupo: Apresentações de Trabalhos. Qty: 91
    7. Pareando por erros quadráticos. Grupo: Atividades de Conselho, Direção ou Administração. Qty: 91
    7. Pareando por 

    7. Pareando por erros quadráticos. Grupo: Vínculos Empregatícios ou Funcionais. Qty: 45
    7. Pareando por erros quadráticos. Grupo: Anos de Doutor.
    Calculando o Desvio Padrão por id pareado
    Incluindo no DB.
16.Fazendo id: 6832614989904988 / 716. Iniciado em 2022-04-18 18:37:08.497572
16. 2.2346368715083798% feitos. Fazendo id: 6832614989904988. Acabará em 2022-04-19 16:01:20.412979
    1. Pareando pela área de conhecimento
    2. Pareando pelo Vínculo
    6. Pareando por região
    7. Pareando por erros quadráticos. Grupo: Apresentações de Trabalhos. Qty: 33617
    7. Pareando por erros quadráticos. Grupo: Atividades de Conselho, Direção ou Administração. Qty: 33617
    7. Pareando por erros quadráticos. Grupo: Atividades de Ensino. Qty: 33617
    7. Pareando por erros quadráticos. Grupo: Atividades de Pesquisa e Desenvolvimento. Qty: 33617
    7. Pareando por erros quadráticos. Grupo: Cursos. Qty: 33617
    7. Pareando por erros quadráticos. Grupo: Doutorados Iniciados. 

...

    2. Pareando pelo Vínculo
    6. Pareando por região
    7. Pareando por erros quadráticos. Grupo: Apresentações de Trabalhos. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Atividades de Conselho, Direção ou Administração. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Atividades de Ensino. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Atividades de Pesquisa e Desenvolvimento. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Cursos. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Doutorados Iniciados. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Eventos. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Graduações Iniciadas. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Mestrados. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Mídias. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Orientações de Doutorado. Qty: 103
    7. Pareando por erros quadráticos. Grupo: Orientações de Graduação. Qty: 103
    7. Par

    2. Pareando pelo Vínculo
    6. Pareando por região
    7. Pareando por erros quadráticos. Grupo: Apresentações de Trabalhos. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Atividades de Conselho, Direção ou Administração. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Atividades de Ensino. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Atividades de Pesquisa e Desenvolvimento. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Doutorados Iniciados. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Eventos. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Graduações Iniciadas. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Livre Docência Obtida. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Orientações - Outras. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Orientações de Doutorado. Qty: 166
    7. Pareando por erros quadráticos. Grupo: Orientações de Graduação. Qty: 166
    7. Pareando por erros quadráticos. Grupo: O

    7. Pareando por erros quadráticos. Grupo: Produções Técnicas. Qty: 144
    7. Pareando por erros quadráticos. Grupo: Publicações de Artigos. Qty: 144
    7. Pareando por erros quadráticos. Grupo: Serviços Técnicos/Especializados. Qty: 144
    7. Pareando por erros quadráticos. Grupo: Vínculos Empregatícios ou Funcionais. Qty: 144
    7. Pareando por erros quadráticos. Grupo: Anos de Doutor.
    Calculando o Desvio Padrão por id pareado
    Incluindo no DB.
708.Fazendo id: 7379950462864398 / 716. Iniciado em 2022-04-18 18:37:08.497572
708. 98.88268156424581% feitos. Fazendo id: 7379950462864398. Acabará em 2022-04-18 21:16:26.066489
    1. Pareando pela área de conhecimento
    2. Pareando pelo Vínculo
    6. Pareando por região
    7. Pareando por erros quadráticos. Grupo: Apresentações de Trabalhos. Qty: 138
    7. Pareando por erros quadráticos. Grupo: Atividades de Conselho, Direção ou Administração. Qty: 138
    7. Pareando por erros quadráticos. Grupo: Atividades de Ensino. Qt

    7. Pareando por erros quadráticos. Grupo: Participação em Congressos. Qty: 490
    7. Pareando por erros quadráticos. Grupo: Pós-Doutorados. Qty: 490
    7. Pareando por erros quadráticos. Grupo: Premiações. Qty: 490
    7. Pareando por erros quadráticos. Grupo: Produções Bibliográficas. Qty: 490
    7. Pareando por erros quadráticos. Grupo: Produções Técnicas. Qty: 490
    7. Pareando por erros quadráticos. Grupo: Publicações de Artigos. Qty: 490
    7. Pareando por erros quadráticos. Grupo: Vínculos Empregatícios ou Funcionais. Qty: 490
    7. Pareando por erros quadráticos. Grupo: Anos de Doutor.
    Calculando o Desvio Padrão por id pareado
    Incluindo no DB.
712.Fazendo id: 7385454242498663 / 716. Iniciado em 2022-04-18 18:37:08.497572
712. 99.4413407821229% feitos. Fazendo id: 7385454242498663. Acabará em 2022-04-18 21:16:04.077420
    1. Pareando pela área de conhecimento
    2. Pareando pelo Vínculo
    6. Pareando por região
    7. Pareando por erros quadráticos. Grupo: 

    7. Pareando por erros quadráticos. Grupo: Vínculos Empregatícios ou Funcionais. Qty: 160
    7. Pareando por erros quadráticos. Grupo: Anos de Doutor.
    Calculando o Desvio Padrão por id pareado
    Incluindo no DB.
716. 100.0% feitos. Último feito: 7387849390654555. Acabou em 2022-04-18 21:15:33.151011.


In [ ]:
financiados

In [ ]:
dt